In [2]:
import numpy as np
import sys, os, shutil
import pandas as pd
import collections as cl
import itertools as it
import timeit
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import copy as cp
import pyprojroot as pr

In [3]:
User=['Emory','WG'][0]
print(User)

#Change directory
if User=='Emory':
    os.chdir(pr.here("./Data/fake"))
elif User == 'WG':
    os.chdir(pr.here("./Data/real"))
os.getcwd()

Emory


'C:\\Users\\yche465\\Desktop\\AIM1\\Codes\\PrEP-Traj-Clustering\\Data\\fake'

### Convert raw data to proportion of moving coverage (PMC) time series

In [9]:
#Import raw dataset
data=pd.read_csv("syndata_13k.csv").astype({'ID':'int64'})


In [10]:
data.iloc[1:5,:]

,ID,PrEP_Start,PrEP_End
1,20,10/25/2023,01/11/2024
2,20,05/17/2024,06/26/2024
3,93,03/25/2023,10/26/2023
4,102,12/22/2023,05/06/2024


In [11]:
#the data type of the column is text string
print(type(data['PrEP_Start'][1]))
print(type(data['PrEP_Start'][2]))

<class 'str'>
<class 'str'>


In [12]:
#convert the column's data type to date time 
data['PrEP_Start']= pd.to_datetime(data['PrEP_Start'])
data['PrEP_End']= pd.to_datetime(data['PrEP_End'])

In [13]:
#the data type of the column is now changed to time stamp
print(type(data['PrEP_Start'][1]))
print(type(data['PrEP_End'][1]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [14]:
#create PrEP initiation date variable
startdate=data.groupby("ID").agg('min').drop(columns=['PrEP_End'])
startdate=startdate.rename(columns={'PrEP_Start':'Initiation'})
data2=data.merge(startdate, on="ID",how='left')
#rename ID variable
fdata=data2.rename(columns={'ID':'PID'})

In [15]:
fdata

,PID,PrEP_Start,PrEP_End,Initiation
0,20,2023-02-26,2023-05-16,2023-02-26
1,20,2023-10-25,2024-01-11,2023-02-26
2,20,2024-05-17,2024-06-26,2023-02-26
3,93,2023-03-25,2023-10-26,2023-03-25
4,102,2023-12-22,2024-05-06,2023-12-22
...,...,...,...,...
19792,191919,2023-11-14,2024-04-07,2023-11-14
19793,191934,2023-08-21,2024-01-14,2023-08-21
19794,191941,2023-02-05,2023-06-28,2023-02-05
19795,191955,2023-08-11,2023-10-20,2023-08-11


In [16]:
#de-identify the date
a=fdata["PrEP_Start"] - fdata["Initiation"] 
fdata['PrEP_start_days']=a.dt.days
a=fdata["PrEP_End"] - fdata["Initiation"] 
fdata['PrEP_end_days']=a.dt.days
fdata=fdata[['PID','PrEP_start_days','PrEP_end_days']]

In [17]:
#Create binary daily intake/coverage matrix (medcover)
id_pool=np.unique(fdata["PID"])
medcover=np.zeros((len(id_pool),365*2))
for k in range(len(id_pool)):
    Rx_start=np.array(fdata.loc[fdata["PID"]==id_pool[k],"PrEP_start_days"])
    Rx_end=np.array(fdata.loc[fdata["PID"]==id_pool[k],"PrEP_end_days"])
    
    for j in range(len(Rx_start)):
        medcover[k][Rx_start[j]:Rx_end[j]+1]=1

In [18]:
#Create PMC matrix: % of the 14 days with medication intake/coverage
start=np.array(range(0,103*7,7))
end=start+14
PMC=np.zeros((len(id_pool),103))
for k in range(len(id_pool)):    
    for i in range(len(start)):
        PMC[k][i]=sum(medcover[k][start[i]:end[i]])/14
#Create binary PrEP sero-protection matrix (0: PMC<0.57, 1: otherwise)  
SeroProtect=np.multiply(PMC>=4/7,1)

In [19]:
#Append subject ID column to the SeroProtect matrix
SeroProtect_indexed=np.column_stack((id_pool,SeroProtect))

In [20]:
#Export PMC matrix
f= open("PMC_13k.txt","w+")
for i in range(len(PMC)):
    for j in range(103):
        if j == 102:
            f.write("%s\n" % (PMC[i][j]))
        else:
            f.write("%s," % (PMC[i][j]))
f.close()

In [21]:
#Export sero-protection matrix
f= open("SeroProtect_13k.txt","w+")
for i in range(len(SeroProtect_indexed)):
    for j in range(104):
        if j == 103:
            f.write("%s\n" % (SeroProtect_indexed[i][j]))
        else:
            f.write("%s," % (SeroProtect_indexed[i][j]))
f.close()